# Images visualizer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import segyio
import os
import dask
import sys
from skimage import exposure

## Datasets loading

In [2]:
class Visualizer():

    def __init__(self,
        data1_file:str,
        data2_file:str,
        border_file:str=None
    ):
        self.data1_file = data1_file
        self.data2_file = data2_file
        self.border_file = border_file 
        
        if self.data1_file[-3:] == 'npy' and self.data2_file[-3:] == 'npy':
            self.dataset1 = np.load(self.data1_file, mmap_mode='r')
            self.dataset2 = np.load(self.data2_file, mmap_mode='r')
        else:
            print('Both data need to be in npy format')
            sys.exit()
        
        #print(np.unique(self.dataset2))
        print(f'Total dimensions dataset 1: {self.dataset1.shape}')
        print(f'Total dimensions dataset 2: {self.dataset2.shape}')

        if self.border_file != None:
            self.border_data = np.load(self.border_file, mmap_mode='r')
            print('Border file loaded successfully')
        
    def seismic_section_plot(self,
                             plot,
                             sections_in,
                             sections_cross,
                             sections_time,
                             section_type,
                             horizons,
                             width,
                             height,
                             cmap1,
                             cmap2,
                             contrast1,
                             contrast2):

        self.plot = plot
        self.sections_in = sections_in
        self.sections_cross = sections_cross
        self.sections_time = sections_time
        self.section_type = section_type
        self.horizons = horizons
        self.width = width
        self.height = height
        self.cmap1 = cmap1
        self.cmap2 = cmap2
        self.contrast1 = contrast1
        self.contrast2 = contrast2

        if self.section_type == 'yz':
            x_label, y_label = 'Y direction', 'Z direction'
            section_to_plot = self.sections_in
            suptitle = f'X section {section_to_plot}'
            plot_data1 = self.dataset1[section_to_plot, :, :].T
            plot_data2 = self.dataset2[section_to_plot, :, :].T
            
            if self.border_file != None:
                plot_border = self.border_data[section_to_plot, :, :].T

        elif self.section_type == 'xz':
            x_label, y_label = 'X direction', 'Z direction'
            section_to_plot = self.sections_cross
            suptitle = f'Y section {section_to_plot}'
            plot_data1 = self.dataset1[:, section_to_plot, :].T
            plot_data2 = self.dataset2[:, section_to_plot, :].T

            if self.border_file != None:
                plot_border = self.border_data[:, section_to_plot, :].T

        elif self.section_type == 'xy':
            x_label, y_label = 'X direction', 'Y direction'
            section_to_plot = self.sections_time
            suptitle = f'Z section {section_to_plot}'
            plot_data1 = self.dataset1[:, :, section_to_plot].T
            plot_data2 = self.dataset2[:, :, section_to_plot].T

            if self.border_file != None:
                plot_border = self.border_data[:, :, section_to_plot].T

        if self.contrast1 == 'Constrasted':
            plot_data1 = self.apply_contrast(plot_data1)
        if self.contrast2 == 'Constrasted':
            plot_data2 = self.apply_contrast(plot_data2)

        if self.border_file != None:
            plot_border = 1 - plot_border
            if plot_border.dtype == int:
                plot_border = plot_border.astype(float)
            plot_border[plot_border == 1] = np.nan
                        
        if self.cmap1 == 'binary (black and white)' or self.cmap1 == 'binary (blue and white)' or self.cmap1 == 'binary (red and white)':
            if self.cmap1 == 'binary (black and white)':
                self.cmap1 = plt.cm.colors.ListedColormap(['white', 'black'])
            if self.cmap1 == 'binary (blue and white)':
                self.cmap1 = plt.cm.colors.ListedColormap(['white', 'blue'])
            if self.cmap1 == 'binary (red and white)':
                self.cmap1 = plt.cm.colors.ListedColormap(['white', 'red'])
            plot_data1 = np.round(plot_data1)
            plot_data1 = plot_data1 != 0 # mask
        
        if self.cmap2 == 'binary (black and white)' or self.cmap2 == 'binary (blue and white)' or self.cmap2 == 'binary (red and white)':
            if self.cmap2 == 'binary (black and white)':
                self.cmap2 = plt.cm.colors.ListedColormap(['white', 'black'])
            if self.cmap2 == 'binary (blue and white)':
                self.cmap2 = plt.cm.colors.ListedColormap(['white', 'blue'])
            if self.cmap2 == 'binary (red and white)':
                self.cmap2 = plt.cm.colors.ListedColormap(['white', 'red'])
            plot_data2 = np.round(plot_data2)
            plot_data2 = plot_data2 != 0 # mask
     
        if self.cmap2 == 'highlight isovalues':
            self.cmap2 = plt.cm.colors.ListedColormap(['white', 'red'])
            plot_data2 = np.round(plot_data2)
            plot_data2 = plot_data2 == self.isovalues # mask
            for point, z in zip(*np.where(plot_data2)):
                plot_data2[point, z:z+10] = True

        if self.plot == 'Plot data 1 and data 2':
            fig, ax = plt.subplots(1, 2, figsize=(self.width, self.height), dpi=500, sharey=True)
            ax[0].imshow(plot_data1, cmap=self.cmap1)
            ax[0].set_ylabel(y_label)
            ax[0].set_xlabel(x_label)
            ax[1].imshow(plot_data2, cmap=self.cmap2)
            ax[1].set_xlabel(x_label)
            if self.border_file != None:
                ax[0].imshow(plot_border, cmap='gray')
                ax[1].imshow(plot_border, cmap='gray')

        elif self.plot == 'Plot data 1':
            fig, ax = plt.subplots(1, 1, figsize=(self.width, self.height), dpi=500, sharey=True)
            ax.imshow(plot_data1, cmap=self.cmap1)
            ax.set_ylabel(y_label)
            ax.set_xlabel(x_label)
            if self.border_file != None:
                ax.imshow(plot_border, cmap='gray')
            
        elif self.plot == 'Plot data 2':
            fig, ax = plt.subplots(1, 1, figsize=(self.width, self.height), dpi=500, sharey=True)
            ax.imshow(plot_data2, cmap=self.cmap2)
            ax.set_ylabel(y_label)
            ax.set_xlabel(x_label)
            if self.border_file != None:
                ax.imshow(plot_border, cmap='gray')

        elif self.plot == 'Sobreposition (data 2 over data 1)':
            fig, ax = plt.subplots(1, 1, figsize=(self.width, self.height), dpi=500, sharey=True)
            ax.imshow(plot_data1, cmap=self.cmap1)
            ax.imshow(plot_data2, cmap=self.cmap2, alpha=0.5)
            ax.set_ylabel(y_label)
            ax.set_xlabel(x_label)
            if self.border_file != None:
                ax.imshow(plot_border, cmap='gray')

        plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.1)
        fig.suptitle(suptitle)

    def data_standardization(self, data):
        # standardization
        mean = np.mean(data)
        std_dev = np.std(data)
        data = (data - mean) / std_dev
        # Clip the outliers
        data = np.clip(data, -3, 3)
    
        min_val = np.min(data)
        max_val = np.max(data)
        
        print(f'Standardized data range: [{min_val} , {max_val}]', flush=True)
        data = 2 * (data - min_val) / (max_val - min_val) - 1
        print(f'Normalized data range: [{np.min(data)}, {np.max(data)}]', flush=True)
    
        return data
    
    def apply_contrast(self, img):
        min_value = np.min(img)
        img = np.nan_to_num(img, nan=min_value)
        p5, p95 = np.percentile(img, (5, 95))
        img = exposure.rescale_intensity(img, in_range=(p5, p95), out_range=(0, 1))
        return img

    def widgets_definition(self):

        self.data_to_plot = widgets.RadioButtons(options=['Plot data 1 and data 2', 'Plot data 1', 'Plot data 2', 'Sobreposition (data 2 over data 1)'],
                                        value='Plot data 1 and data 2',
                                        description='Plot:',
                                        disabled=False)

        self.slider_inline = widgets.IntSlider(min=0,
                                          max=self.dataset1.shape[0]-1,
                                          value=0,
                                          step=1,
                                          description='x')

        self.slider_crossline = widgets.IntSlider(min=0,
                                             max=self.dataset1.shape[1]-1,
                                             value=0,
                                             step=1,
                                             description='y')

        self.slider_timeslice = widgets.IntSlider(min=0,
                                            max=self.dataset1.shape[2]-1,
                                            value=0,
                                            step=1,
                                            description='z')

        self.section_view = widgets.Dropdown(options=['yz', 'xz', 'xy'],
                                        value='yz',
                                        description='View plan:',
                                        disabled=False)
        
        self.input_horizons = widgets.IntText(value=1,
                                description='Horizon:',
                                disabled=False)

        self.figsize_width = widgets.FloatText(value=7,
                                        description='Fig width:',
                                        disabled=False)

        self.figsize_height = widgets.FloatText(value=4,
                                        description='Fig height:',
                                        disabled=False)

        self.colormap1 = widgets.Dropdown(options=['gray', 'seismic', 'RdGy', 'viridis', 'plasma', 'inferno', 'magma', 'cividis', 'hsv', 'Paired',
                                              'RdYlBu', 'RdYlGn', 'binary (black and white)', 'binary (blue and white)', 'binary (red and white)'],
                                        value='gray',
                                        description='Colormap 1:',
                                        disabled=False)

        self.colormap2 = widgets.Dropdown(options=['gray', 'seismic', 'RdGy', 'viridis', 'plasma', 'inferno', 'magma', 'cividis', 'hsv', 'Paired',
                                              'RdYlBu', 'RdYlGn', 'binary (black and white)', 'binary (blue and white)', 'binary (red and white)', 'highlight isovalues'],
                                        value='gray',
                                        description='Colormap 2:',
                                        disabled=False)

        self.contrast_img1 = widgets.Dropdown(options=['Raw', 'Constrasted'],
                                        value='Raw',
                                        description='Constrast 1:',
                                        disabled=False)

        self.contrast_img2 = widgets.Dropdown(options=['Raw', 'Constrasted'],
                                value='Raw',
                                description='Constrast 2:',
                                disabled=False)


        self.widgets_list = [self.data_to_plot, self.slider_inline, self.slider_crossline, self.slider_timeslice, self.section_view,
                             self.input_horizons, self.figsize_width, self.figsize_height, self.colormap1, self.colormap2, self.contrast_img1, self.contrast_img2]

        for wid in self.widgets_list:
            wid.layout.display = "none"
            
    def interface(self):
        
        self.widgets_definition()
        
        widgets.interact(self.seismic_section_plot,
                        plot=self.data_to_plot,
                        sections_in=self.slider_inline, 
                        sections_cross=self.slider_crossline,
                        sections_time=self.slider_timeslice,
                        section_type=self.section_view,
                        horizons=self.input_horizons,
                        width=self.figsize_width,
                        height=self.figsize_height,
                        cmap1=self.colormap1,
                        cmap2=self.colormap2,
                        contrast1=self.contrast_img1,
                        contrast2=self.contrast_img2)

        for wid in self.widgets_list:
            wid.layout.display = "visible"

        display(widgets.HBox([self.slider_inline, self.slider_crossline, self.slider_timeslice]))
        display(widgets.HBox([self.section_view, self.input_horizons, self.figsize_width, self.figsize_height]))
        display(widgets.HBox([self.colormap1, self.colormap2, self.contrast_img1, self.contrast_img2]))
        display(widgets.HBox([self.data_to_plot]))

In [5]:
data1_file = '/pgeoprj/godeep/ewac_2/seismic_patterns/datasets/public_real_data/penobscot_dataset.npy'
#data1_file = '/pgeoprj/godeep/ewac_2/seismic_patterns/datasets/public_real_data/parihaka_label.npy'
data2_file = '/pgeoprj/godeep/ewac_2/seismic_patterns/datasets/public_real_data/penobscot_boundaries.npy'
#data1_file = '/pgeoprj/godeep/ewac_2/seismic_patterns/datasets/models_predictions/parihaka_9d_boundaries_prediction.npy'
#data2_file = '/pgeoprj/godeep/ewac_2/seismic_patterns/datasets/models_predictions/parihaka_9d_boundaries_prediction.npy'
#data2_file = '/pgeoprj/godeep/ewac_2/seismic_patterns/datasets/models_predictions/parihaka_10a_boundaries_prediction.npy'

view = Visualizer(data1_file, data2_file, border_file=None)

view.interface()

Total dimensions dataset 1: (601, 481, 1501)
Total dimensions dataset 2: (601, 481, 1501)


interactive(children=(RadioButtons(description='Plot:', layout=Layout(display='none'), options=('Plot data 1 a…